# Setting relevant contact parameters in DEM simulations

## I. Case of two spherical particles

### a. Model definiton

In DEM simulations, the forces and torques are assessed from contact laws. In our case, and following the foundation paper of Cundall & Strack [1], we adopt a Linear Spring-Dashpot Model (LSD):

![LSDmodel](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTc2rXAwpiaUpF4_jcB75bRQG9ndg-TupcLWDJBapgisn1BotCK-A)

Let us quickly write the forces experienced by two colliding particles of mass $m_i$ amd $m_j$ respectively, and radius $R_i$ and $R_j$ respectively:

\begin{equation}
F_n = k_n \delta_n - \gamma_n \frac{\partial \delta_n}{\partial t}
\end{equation}

\begin{equation}
F_t = k_t \int_{t_0}^{t} \delta_t(s) ds - \gamma_t \frac{\partial \delta_t}{\partial t}
\end{equation}

where we have called the normal force $F_n$, the normal spring stiffness $k_n$, and normal damping coefficient $\gamma_n$; and where we have denoted their tangential counterpart $F_t$, $k_t$ and $\gamma_t$, and the sliding coefficient $\mu_c$.

That is already a lot of parameters $-$ and we only have considered translation so far. We now have the difficult task to set meaningful values to all of those coefficient.

### b. Model parameters from physical considerations

From Hooke's law and [Hertz theory](https://en.wikipedia.org/wiki/Contact_mechanics#Classical_solutions_for_non-adhesive_elastic_contact), and following [2] (eq. 43), we can relate some particle properties to the contact model parameters:

\begin{equation}
k_n = \frac{4}{3} \frac{E_i E_j}{E_i(1-\sigma_j^2) + E_j(1-\sigma_i^2)} R_{eff}
\end{equation}

or, for two particles of the same material:

\begin{equation}
k_n = \frac{2ER_{eff}}{3(1-\sigma^2)}
\end{equation}

where $R_{eff}=\frac{R_i R_j}{R_i +R_j}$ is the effective radius, $E_i$ is the Young's modulus of the material $i$ and $\sigma_i$ its Poisson ratio.

Then, for two spheres of the same radius $R$ colliding at the relative velocity $v_0$, it is not difficult to show that 
\begin{equation}
e_n = e^{- \gamma_n \frac{\pi}{\sqrt{\omega_0^2 - \gamma_n^2}}}
\end{equation}
where $e_n$ is the normal restitution coefficient and $\omega_0^2=\frac{k_n}{m_{eff}}=\frac{2 k_n}{m}$ is the resonance pulsation of the system, with $m_{eff}=\frac{m_i m_j}{m_i + m_j}$ the effective mass of the system - for simplicity we consider $m_i=m_j=m$. We could also express the latter in terms of $\gamma_n$:
\begin{equation}
\gamma_n = -\frac{\omega_0 ln(e_n)}{\sqrt{\pi^2 + (ln(e_n))^2}}
\end{equation}
The time step $\Delta t$ should be set according to the contact time $T_c$. As summarized in [3], the number of recommended time steps per contact varies strongly from author to author and are often precognized regardless of the integration scheme. However, in [3] it is found that having 20 time steps or more per contact is sufficient to get a resolved enough simulation in general. In our simple two-spheres model, the contact time is expressed as:
\begin{equation}
T_c = \frac{\pi}{\sqrt{\omega_0^2 - \gamma_n^2}} = \sqrt{\frac{m_{eff}(\pi^2 + ln(e_n)^2)}{k_n}}
\end{equation}


Now, let's take some typical values for the material properties and take a look at the stiffness coefficient $k_n$. For steel, typically $E = 210$ GPa and $\sigma = 0.3$. For two 1mm steel balls, we therefore have $k_n = 6.37 \cdot 10^8 N/m$. This means that $T_c \approx 5 \cdot 10^{-7}$s, imposing $\Delta t \leq 2.5 \cdot 10^{-8}$s. For computational reasons, such a small time step is practically unachievable.

At this stage, performing reliable DEM simulations can seem out of reach. However, it has been shown [5,6] that a physical value for the normal stiffness $k_n$ does not play a significant role in the overall dynamics of the system, and that decreasing its value by orders of magnitudes $-$ depending on the system $-$ can be safely performed. In fact, it is not the actual stiffness that drives the behavior of the granular medium, but rather the extent of overlap between the colliding particles. [reference needed here].

### c. Routine to find an admissible set of parameters

We have seen that setting DEM parameters according to the material properties of the particles leads to unfeasible computations and is not the right approach. Since the particles' overlap plays a more significant role than the stiffness, we will set the model parameters according to it:

**1. Set value for the particles overlap $\delta_{max}$, the restitution coefficien $e_n$ and the Coulomb static friction coefficient $\mu_c$:**

It has been stated in [6] and shown in [5] that an appropriate particle overlap should remain between 0.1% and 0.5% of the particle radius. In the following, we will take a particle overlap $\delta_{max} = 3 \cdot 10^{-3} \cdot R$. The restitution coefficient $e_n$ can be set in the range from 0 to 1, and its value is easy to measure experimentally. The sliding friction coefficient is also straightforward to measure experimentally and typically ranges from 0 to 1.5. In the following, we take $e_n = 0.85$ and $\mu_c = 0.5$.


**2. Compute $k_n$ from an estimated impact velocity $v_0$:**

Solving analytically for the overlap $\delta$ and computing its maximum value leads to $\delta_{max} = \frac{v_0}{\omega} e^{-\gamma_n T_{max}} sin\left(\omega T_{max}\right)$ with $\omega = \sqrt{\omega_0^2 - \gamma_n^2}$ and $T_{max} = \frac{1}{\omega} arctan \left(\frac{\omega}{\gamma_n}\right)$. Note that $\omega = \omega(k_n)$. Solving for $k_n$ leads to:


\begin{equation}
k_n = m_{eff} \left( \frac{v_0}{\delta_{max}\sqrt{1+B}} e^{\frac{arctan(A)}{A}} sin(arctan(A)) \right)^2 \\
\text{with} \quad B=-\frac{ln(e_n)}{\sqrt{\pi^2 + ln(en)^2}} \quad \text{and} \quad A=\frac{\sqrt{1+B}}{B}
\end{equation}

**3. Compute the contact time $T_c$ and the normal damping coefficient $\gamma_n$ from eq (7) and eq (6)**

**4. Compute the tangential parameters $k_t$ and $\gamma_t$:**

From Pournin [7] (eq. (22)), we can link the tangential model parameters in the following way:
\begin{equation}
k_t = \frac{1}{T_c^2 \left( \frac{1}{m_{eff}} + \frac{R_1^2}{I_1} + \frac{R_2^2}{I_2} \right)} (\pi^2 + ln(e_t)^2) = \frac{k_n}{1 + m_{eff}\left(\frac{R_1^2}{I_1} + \frac{R_2^2}{I_2} \right)} \frac{\pi^2 + ln(e_t)^2}{\pi^2 + ln(e_n)^2}
\end{equation}

\begin{equation}
\gamma_t = - \frac{2}{\frac{1}{m_{eff}} + \frac{R_1^2}{I_1} + \frac{R_2^2}{I_2}} ln(e_t)
\end{equation}

### d. Contact model in rotation

In the case of spherical particles $-$ or more generally for non-angular particles $-$ it is necessary to add a rolling friction model on top of the translational tangential friction.

## Routines for relevant parameters
### Compute an admissible set of parameters

In [2]:
from math import *

In [16]:
# Input parameters
max_relative_overlap = 0.005 # Here we set the maximum relative overlap to 0.5% of the radius of the particles
e_n = 0.85
g = 9.81
mu_c = 0.5
e_t = 0.6
radius = 0.001
density = 2500.
individual_mass = density * 4./3 * pi * radius**3
mass = individual_mass/2 # the equivalent mass in the case the two particles have the same mass
I = 2./5 * (2*mass) * radius**2 # if the particles are spherical
L = 0.1 # The maximum height from which a particle will be dropped from
v_max = sqrt(g*L) # the expected maximum impact velocity
print('v_max =',v_max,'m/s')


### Set routine starts here
# First we compute the normal stiffness k_n and the normal damping coefficient
delta_max = max_relative_overlap * radius
B = -log(e_n) / sqrt(pi**2 + log(e_n)**2)
A = sqrt(1+B)/B
k_n = mass * ( v_max/(delta_max*sqrt(1+B)) * exp(atan(A)/A) * sin(atan(A)) )**2
print('k_n =',k_n,'N/m')
eta_n = - log(e_n) * sqrt(k_n/mass) / sqrt(pi**2 + log(e_n)**2)
print('eta_n =',eta_n,'s-1')

# Then we compute the tangential stiffness k_t and the tangential damping coefficient eta_t
k_t = k_n * ((pi**2 + log(e_t)**2)/((pi**2 + log(e_n)**2)*(1 + 2*mass*radius**2/I)))
print("k_t =",k_t,"N/m")
Tc = sqrt(mass*(pi**2 + log(e_n)**2)/k_n)
eta_t = -2 * log(e_t)/(Tc*(1/mass + 2*radius**2/I))
print("eta_t =",eta_t,"s-1")

# If applicable, we also compute the rolling stiffness and damping coefficients k_r and C_r
k_r = k_t * radius**2
print("k_r =",k_r,"N/m")
eta_r = 0.3
C_rcritical = 2 * sqrt(I*k_r)
C_r = eta_r * C_rcritical
print("eta_r =",eta_r,"s-1")

# Finally, we compute the time step
delta_t = Tc/20
print("Tc =",Tc,"s")
print("Delta_t =",delta_t,"s")
print("---------------")
print("meff =", mass)
print("pseudo mass =", 1/(1/mass + 2*radius**2/I))
print("-2*mass*log(e_n)/Tc =",-2*mass*log(e_n)/Tc)
print("-2*pseudo_mass*log(e_t)/Tc =",eta_t)

v_max = 0.9904544411531507 m/s
k_n = 227133.774818251 N/m
eta_n = 10760.074768924807 s-1
k_t = 66433.35037879189 N/m
eta_t = 0.10119162817235337 s-1
k_r = 0.06643335037879189 N/m
eta_r = 0.3 s-1
Tc = 1.5103884776631017e-05 s
Delta_t = 7.551942388315508e-07 s
---------------
meff = 5.235987755982989e-06
pseudo mass = 1.4959965017094255e-06
-2*mass*log(e_n)/Tc = 0.11267923948710357
-2*pseudo_mass*log(e_t)/Tc = 0.10119162817235337


### Check if a set of parameters is admissible

In [3]:
# Input parameters
max_relative_overlap = 0.005 # the maximum overlap should not exceed 0.5% of the particle's radius
e_n = 0.45
mu_c = 0.5
e_t = 0.45
radius = 0.001
density = 2500.
mass = 4./3 * pi * radius**3 
I = 2./5 * mass * radius**2

References:

[1] Cundall & Strack

[2] Dviugys & Peters

[3] [Kruggel-Emeden et al., Selection of an appropriate time integration scheme for the discreteelement method (DEM), Computers and Chemical Engineering, 2008](https://www.sciencedirect.com/science/article/pii/S0098135407002864)

[4] [Wachs et al., Grains3D, a flexible DEM approach for particles of arbitrary convex shape — Part I: Numerical model and validations, Powder Technology, 2012.](https://www.sciencedirect.com/science/article/pii/S003259101200191X)

[5] [Yan & Wilkinson, Discrete element modelling (DEM) input parameters: understanding their impact on model predictions using statistical analysis, Computational Particle Mechanics, 2015](https://link.springer.com/article/10.1007/s40571-015-0056-5)

[6] [Cleary, DEM prediction of industrial and geophysical particle flows, Particuology, 2010](https://reader.elsevier.com/reader/sd/pii/S1674200109001308?token=D4745DC00931B53EDA4D83481B70210DF641BBA578B8A1F658ADE8D0A36973394D90F43D2C791309AB5AF0DFD4F20505)

[7] [Pournin & Liebling, Molecular-dynamics force models for better control of energy dissipation in numerical simulationsof dense granular media, Pys. Rev. E, 2001](https://journals-aps-org.ezproxy.library.ubc.ca/pre/pdf/10.1103/PhysRevE.65.011302)